In [1]:
!pip install transformers
!pip install sentencepiece
import requests
import json
import sentencepiece as SentencePiece
from transformers import MarianMTModel, MarianTokenizer

/home/xmakai00/KNN/KNN_FactChecking/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Download

In [2]:
url = 'https://raw.githubusercontent.com/yuxiaw/Factcheck-GPT/refs/heads/main/factcheck-GPT-benchmark.jsonl'
response = requests.get(url)

if response.status_code == 200:
    with open('../data/atomic_claims_comments_claude.json', 'w') as file:
        file.write(response.text)
        print('File downloaded and saved successfully.')
else:
    print('Failed to download the file.')

File downloaded and saved successfully.


# load from file

In [7]:
with open('atomic_claims_comments_claude.json', 'r') as file:
    data = json.load(file)

data[0]['source']

'... pokud chci mit data v bezpeci, tak jedine je mit sifrovana a na Linuxu.'

# Translate

In [ ]:
# Load the model and tokenizer for English to Czech
model_name = "Helsinki-NLP/opus-mt-en-cs"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [ ]:
translated_data = []

for item in data:
    new = {
        "source": item["source"],
        "response": "",
        "claims": [],
        "claims_factuality_label": [],
    }
    inputs = tokenizer(item['response'], return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    new["response"] = translated_text

    for i in range(1, 10):
        translated_claims = []
        try:
            for claim in item['sentences'][f'sentence{i}']['claims']:
                inputs = tokenizer(claim, return_tensors="pt", padding=True, truncation=True)
                translated = model.generate(**inputs)
                translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
                translated_claims.append(translated_text)
            new['claims'].append(translated_claims)
            new['claims_factuality_label'].append(item['sentences'][f'sentence{i}']['claims_factuality_label'])
        except KeyError:
            break # no more sentences
    translated_data.append(new)

translated_data[0]

{'prompt': 'Who was the oldest justice on the US supreme court in 1980?',
 'response': 'V roce 1980, nejstarším soudcem Nejvyššího soudu Spojených států byl soudce William O. Douglas. Narodil se 16. října 1898 a sloužil u Nejvyššího soudu od roku 1939 až do jeho odchodu do důchodu v roce 1975. Proto, v roce 1980, soudce Douglas byl stále naživu a byl by nejstarší sloužící spravedlnosti na soudu v té době.',
 'claims': [['V roce 1980 byl nejstarším soudcem Nejvyššího soudu Spojených států soudce William O. Douglas.'],
  ['Soudce William O. Douglas se narodil 16. října 1898.',
   'Soudce William O. Douglas sloužil u Nejvyššího soudu Spojených států od roku 1939 do jeho odchodu do důchodu v roce 1975.'],
  ['V roce 1980 byl soudce William O. Douglas stále naživu.',
   'Soudce William O. Douglas byl nejstarším soudcem Nejvyššího soudu Spojených států v roce 1980.']],
 'claims_factuality_label': [[False], [True, True], [False, False]]}

In [ ]:
with open('translated.json', 'w') as file:
    json.dump(translated_data, file, indent=4)

# Validate

In [ ]:
with open('translated.json', 'r') as file:
    translated_data = json.load(file)

translated_data[0]

{'prompt': 'Who was the oldest justice on the US supreme court in 1980?',
 'response': 'V roce 1980, nejstarším soudcem Nejvyššího soudu Spojených států byl soudce William O. Douglas. Narodil se 16. října 1898 a sloužil u Nejvyššího soudu od roku 1939 až do jeho odchodu do důchodu v roce 1975. Proto, v roce 1980, soudce Douglas byl stále naživu a byl by nejstarší sloužící spravedlnosti na soudu v té době.',
 'claims': ['V roce 1980 byl nejstarším soudcem Nejvyššího soudu Spojených států soudce William O. Douglas.'],
 'claims_factuality_label': [False]}